## Goal:
Map as many bookings (“rows”) as possible over 10 years. Approach: Embed combinations of columns like “Zweckbestimmung” and make a vector database out of them. This is done for one year (here 2023). All other years are vectorized (embedded) in the same way and mapped to the next vectors of the database (reference year 2023).

## Issues:
1. Encoding error UTF 8 for different data processing (formats, programs). e.g. "\xa0" non-breaking space can interfere with string matching. (should be carried out in data cleaning)
2. Not all encoder models understand German or are suitable for the task.
3. Model cannot process numbers so well. 
4. Each posting is assigned the e.g. 5 closest entities with a confidence [(1, 0.8), (2, 0.7), (3, 0.6)...]. If you blindly choose the first match, then conflicts arise, e.g. an entity has several matches. 

## Solutions: 
1. Normalize relevant str columns. Either with unicodedata.normalize or with pd.Series.str.normalize.
2. Use recommended model for embeddings (https://neuml.github.io/txtai/models/)
3. Map all numbers to their spelled out words and embed again.
4. Match only if two conditions are met: the reference entity has not yet been matched and the confidence level is greater than 0.8. In addition, all tables were initially sorted by the largest entities (in €) so that large positions are matched first.

Summarized: With the first method of merging a unique index, 1k rows of approx. 6k rows were matched. With the NLP approach, up to 3k rows can be matched.\
Limitations: Not all rows find a match (about 2k without match) and sometimes there is none. There is a tradeoff between false matches and no matches, where false positive (wrong match) is worse than false negative (no match).\
**Even large LLMs (decoder only) would probably have problems matching “Arbeitslosengeld II” and “Bürgergeld” without context.*


Further notes: On jupyter notebooks, the kernels died during embedding. That's why I first tried out some models (mainly encoders) on the cluster. But the model recommended by txtai beats all BERT_german or BERT_german_finance and others. And since it is still an encoder (based on https://huggingface.co/microsoft/MiniLM-L12-H384-uncased “2.7x faster than BERT-Base”) I decided to continue locally but do the embeddings in a python file.

In [2]:
import numpy as np
import pandas as pd
from time import time
import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Functions

In [3]:
## Functions
def drop_nan(df):
    col_ist = [col for col in df.columns if col.startswith("Ist")][0]
    df = df.dropna(subset=col_ist, ignore_index=True)
    df = df[df[col_ist] > 1000]
    df[["Epl.", "Kap.", "Tit."]] = df[["Epl.", "Kap.", "Tit."]].astype(int, copy=False)
    df = df[["Epl.", "Kap.", "Tit.", col_ist, "Zweckbestimmung"]]
    return df

def make_id_col(df, zahlen_dict):
    df["id_help"] = df["Tit."].astype(str).str.zfill(5) + df["Epl."].astype(str).str.zfill(2) + df["Kap."].astype(str).str.zfill(2)
    df["id"] = df["id_help"].apply(map_numbers) + df["Zweckbestimmung"]
    #df["id"] = df["Zweckbestimmung"].astype(str) +" "+ df["Tit."].astype(str).str.zfill(5) + df["Epl."].astype(str).str.zfill(2) + df["Kap."].astype(str).str.zfill(2)
    #df = df.drop(["Epl.", "Kap.", "Tit."], axis=1)
    return df

def pipeline(range_years):
    df_result = pd.read_excel(f"data/HR20{range_years[0]}.xlsx")
    df_result = make_id_col(drop_nan(df_result))[["id", "Zweckbestimmung", f"Ist 20{range_years[0]}"]]
    for year in range(range_years[0]+1, range_years[1]):
        df_i = pd.read_excel(f"data/HR20{year}.xlsx")
        df_i = drop_nan(df_i)
        df_i = make_id_col(df_i).drop("Zweckbestimmung", axis=1)
        df_result = pd.merge(df_result, df_i, on='id', how="inner")
    return df_result

def map_numbers(string):
    string = " ".join([zahlen_dict[int(dig)] for dig in str(string)]) + " "
    return string

In [4]:
# make number mapper
zahlen = "null, eins, zwei, drei, vier, fünf, sechs, sieben, acht, neun".split(", ")
zahlen_dict = { k:v for k,v in zip(range(10), zahlen)}

# pipline for every year

In [4]:
## Get data
df_test = pd.read_excel("data/HR2012.xlsx")
df_test_2 = pd.read_excel("data/HR2023.xlsx")

df_test = drop_nan(df_test)
df_test = drop_nan(df_test)
df_test = make_id_col(df_test, zahlen_dict)

df_test_2 = drop_nan(df_test_2)
df_test_2 = drop_nan(df_test_2)
df_test_2 = make_id_col(df_test_2, zahlen_dict) #.sort_values("Ist 2022")

# idx_arbeit = df_test[df_test["Zweckbestimmung"] == "Arbeitslosengeld II"].index[0]
# id_idx_arbeit = df_test.iloc[idx_arbeit,-1]
# print(idx_arbeit, id_idx_arbeit)

id_idx_arbeit = df_test["id"].to_list()
print(id_idx_arbeit[:2])

c:\Users\sebas\Documents\Data_Science\WS_24_25\Urban_technologie\project\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['zwei drei zwei null eins null eins null eins Beteiligung der Länder an der Deutschen Künstlerhilfe und sonstige ihr zugedachte Einnahmen', 'vier zwei eins null eins null eins null eins Bezüge des Bundespräsidenten']


In [6]:
# Load embedded df
df_emb = pd.read_csv("data/vergleich_HR2012_s.csv")

In [7]:
df_emb

,Unnamed: 0,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,id_help,id,mapped_idx
0,5276,60,1,1501,7.608407e+10,Umsatzsteuer,15016001,nulleinsfünfnulleinssechsnullnulleins Umsatzst...,Umsatzsteuer
1,5272,60,1,1101,6.313621e+10,Lohnsteuer,11016001,nulleinseinsnulleinssechsnullnulleins Lohnsteuer,Lohnsteuer
2,5281,60,1,3103,3.522622e+10,"Energiesteuer (sonstiges Aufkommen, ohne das i...",31036001,nulldreieinsnulldreisechsnullnulleins Energies...,"Energiesteuer (sonstiges Aufkommen, ohne das i..."
3,3078,11,13,63681,3.156989e+10,Zuschuss des Bundes an die allgemeine Rentenve...,636811113,sechsdreisechsachteinseinseinseinsdrei Zuschus...,Zuschuss des Bundes an die allgemeine Rentenve...
4,5277,60,1,1601,2.788054e+10,Einfuhrumsatzsteuer,16016001,nulleinssechsnulleinssechsnullnulleins Einfuhr...,Einfuhrumsatzsteuer
...,...,...,...,...,...,...,...,...,...
5396,838,6,7,81202,1.099070e+03,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...",812020607,achteinszweinullzweinullsechsnullsieben Erwerb...,"Erwerb von Geräten, Ausstattungs- und Ausrüstu..."
5397,1436,6,67,45357,1.084700e+03,"Trennungsgeld, Fahrtkostenzuschüsse sowie Umzu...",453570667,vierfünfdreifünfsiebennullsechssechssieben Tre...,"Trennungsgeld, Fahrtkostenzuschüsse sowie Umzu..."
5398,1956,8,14,67101,1.076000e+03,Aufwendungen für Besucherfahrten zu Gräbern un...,671010814,sechssiebeneinsnulleinsnullachteinsvier Aufwen...,Aufwendungen für Gräber der Opfer von Krieg un...
5399,1406,6,40,16204,1.034430e+03,Zinsen aus Darlehen an Deutsche aus dem in Art...,162040640,einssechszweinullviernullsechsviernull Zinsen ...,Tilgung aus Darlehen an Deutsche aus dem in Ar...


In [8]:
df_emb[df_emb["Zweckbestimmung"] != df_emb["mapped_idx"]]

,Unnamed: 0,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,id_help,id,mapped_idx
2,5281,60,1,3103,3.522622e+10,"Energiesteuer (sonstiges Aufkommen, ohne das i...",31036001,nulldreieinsnulldreisechsnullnulleins Energies...,"Energiesteuer (sonstiges Aufkommen, ohne das i..."
8,3058,11,12,68112,1.895134e+10,Arbeitslosengeld II,681121112,sechsachteinseinszweieinseinseinszwei Arbeitsl...,Entgelte der Arbeitnehmerinnen und Arbeitnehmer
11,4267,15,2,63606,1.400000e+10,Pauschale Abgeltung der Aufwendungen der Krank...,636061502,sechsdreisechsnullsechseinsfünfnullzwei Pausch...,Pauschale Abgeltung der Aufwendungen der Krank...
14,5274,60,1,1301,1.002753e+10,Nicht veranlagte Steuern vom Ertrag (ohne das ...,13016001,nulleinsdreinulleinssechsnullnulleins Nicht ve...,Nicht veranlagte Steuern vom Ertrag (ohne das ...
16,5339,60,2,83624,8.686848e+09,Beteiligung am Grundkapital des Europäischen S...,836246002,achtdreisechszweiviersechsnullnullzwei Beteili...,Zuschüsse zu Vorhaben zur Förderung des europä...
...,...,...,...,...,...,...,...,...,...
5383,1071,6,17,68501,1.240000e+03,Mitgliedsbeiträge an Vereine und Gesellschaften,685010617,sechsachtfünfnulleinsnullsechseinssieben Mitgl...,Zuschuss an die Kulturpolitische Gesellschaft
5389,2702,10,15,52603,1.175700e+03,Ausgaben für Mitglieder von Fachbeiräten und ä...,526031015,fünfzweisechsnulldreieinsnulleinsfünf Ausgaben...,"Sachverständige, Ausgaben für Mitglieder von F..."
5396,838,6,7,81202,1.099070e+03,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...",812020607,achteinszweinullzweinullsechsnullsieben Erwerb...,"Erwerb von Geräten, Ausstattungs- und Ausrüstu..."
5398,1956,8,14,67101,1.076000e+03,Aufwendungen für Besucherfahrten zu Gräbern un...,671010814,sechssiebeneinsnulleinsnullachteinsvier Aufwen...,Aufwendungen für Gräber der Opfer von Krieg un...


In [12]:
df_different = df_emb[df_emb["Zweckbestimmung"] != df_emb["mapped_idx"]]

In [23]:
for i in range(0,13):
    print(repr(df_different.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different.iloc[i]["mapped_idx"])+"\n")

'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)' ,
'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)'

'Arbeitslosengeld II' ,
'Entgelte der Arbeitnehmerinnen und Arbeitnehmer'

'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamt gesellschaftliche Aufgaben (Gesundheitsfonds)' ,
'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamtgesellschaftliche Aufgaben (Gesundheitsfonds)'

'Nicht veranlagte Steuern vom Ertrag (ohne das in Tit. 018 03 erfasste Aufkommen)' ,
'Nicht veranlagte Steuern vom Ertrag (ohne das in Tit.\xa0018\xa003 erfasste Aufkommen)'

'Beteiligung am Grundkapital des Europäischen Stabilitätsmechanismus (ESM)' ,
'Zuschüsse zu Vorhaben zur Förderung des europäischen Gedankens'

'Beteiligung des Bundes an den Kosten der Arbeitsförderung' ,
'Erstattung von Verwaltungskosten an die Bundesagentur für Arbeit für die Durchführung des Bundeskindergeld

In [26]:
df_emb2 = pd.read_csv("data/vergleich_HR2012_s2_80.csv")
df_emb2

,Unnamed: 0,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,id_help,id,mapped_idx
0,5276,60,1,1501,7.608407e+10,Umsatzsteuer,15016001,nulleinsfünfnulleinssechsnullnulleins Umsatzst...,Umsatzsteuer
1,5272,60,1,1101,6.313621e+10,Lohnsteuer,11016001,nulleinseinsnulleinssechsnullnulleins Lohnsteuer,Lohnsteuer
2,5281,60,1,3103,3.522622e+10,"Energiesteuer (sonstiges Aufkommen, ohne das i...",31036001,nulldreieinsnulldreisechsnullnulleins Energies...,"Energiesteuer (sonstiges Aufkommen, ohne das i..."
3,3078,11,13,63681,3.156989e+10,Zuschuss des Bundes an die allgemeine Rentenve...,636811113,sechsdreisechsachteinseinseinseinsdrei Zuschus...,Zuschuss des Bundes an die allgemeine Rentenve...
4,5277,60,1,1601,2.788054e+10,Einfuhrumsatzsteuer,16016001,nulleinssechsnulleinssechsnullnulleins Einfuhr...,Einfuhrumsatzsteuer
...,...,...,...,...,...,...,...,...,...
5396,838,6,7,81202,1.099070e+03,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...",812020607,achteinszweinullzweinullsechsnullsieben Erwerb...,"Erwerb von Geräten, Ausstattungs- und Ausrüstu..."
5397,1436,6,67,45357,1.084700e+03,"Trennungsgeld, Fahrtkostenzuschüsse sowie Umzu...",453570667,vierfünfdreifünfsiebennullsechssechssieben Tre...,NaN
5398,1956,8,14,67101,1.076000e+03,Aufwendungen für Besucherfahrten zu Gräbern un...,671010814,sechssiebeneinsnulleinsnullachteinsvier Aufwen...,NaN
5399,1406,6,40,16204,1.034430e+03,Zinsen aus Darlehen an Deutsche aus dem in Art...,162040640,einssechszweinullviernullsechsviernull Zinsen ...,NaN


In [25]:
df_emb2 = pd.read_csv("data/vergleich_HR2012_s2_80.csv")
df_different2 = df_emb2[df_emb2["Zweckbestimmung"] != df_emb2["mapped_idx"]]
print(df_emb2.shape[0], df_emb2["mapped_idx"].isna().sum())
for i in range(0,9):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

5401 2112
'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)' ,
'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)'

'Arbeitslosengeld II' ,
nan

'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamt gesellschaftliche Aufgaben (Gesundheitsfonds)' ,
'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamtgesellschaftliche Aufgaben (Gesundheitsfonds)'

'Nicht veranlagte Steuern vom Ertrag (ohne das in Tit. 018 03 erfasste Aufkommen)' ,
'Nicht veranlagte Steuern vom Ertrag (ohne das in Tit.\xa0018\xa003 erfasste Aufkommen)'

'Beteiligung am Grundkapital des Europäischen Stabilitätsmechanismus (ESM)' ,
nan

'Beteiligung des Bundes an den Kosten der Arbeitsförderung' ,
'Erstattung von Verwaltungskosten an die Bundesagentur für Arbeit für die Durchführung des Bundeskindergeldgesetzes'

'Einnahmen aus der streckenbezogenen LKW-Maut' ,
'Einnahmen aus der streckenbezogenen 

In [9]:
print('Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)')
print('Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)')
print(r'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)')

Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)
Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)
Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)


In [ ]:
'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)' \
    == 'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)'

True

In [ ]:
'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)' \
    == 'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)'.replace("\xa0", " ")

In [11]:
import unicodedata

In [12]:
new_str = unicodedata.normalize("NFKD",'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)')

In [13]:
new_str

'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)'

In [5]:
for i in range(9,18):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Beteiligung des Bundes an der Grundsicherung im Alter und bei Erwerbsminderung' ,
'Erstattungen des Bundes für die Grundsicherung im Alter und bei Erwerbsminderung'

'Bilaterale Finanzielle Zusammenarbeit' ,
'Zinsen aus Darlehen der bilateralen Finanziellen Zusammenarbeit und Erträge aus Treuhandbeteiligungen'

'Erhöhung des Kapitalanteils an der Europäischen Investitionsbank' ,
nan

'Kernbrennstoffsteuer' ,
nan

'Hochschulpakt 2020' ,
nan

'Kompensationszahlungen an die Länder wegen Beendigung der Finanzhilfen des Bundes für Investitionen zur Verbesserung der Verkehrsverhältnisse der Gemeinden' ,
nan

'Zinsen gem. § 4 des Schlusszahlungsfinanzierungsgesetzes (SchlussFinG)' ,
'Zinsen gem. §\xa04 des Schlusszahlungsfinanzierungsgesetzes (SchlussFinG)'

'Baukostenzuschüsse für Investitionen in die Schienenwege der Eisenbahnen des Bundes' ,
'Baukostenzuschüsse für Investitionen in die Schienenwege der nicht bundeseigenen Eisenbahnen'

'Zuschüsse an die Träger der Krankenversicherung der 

In [33]:
df_emb2 = pd.read_csv("data/vergleich_HR2012_s2_60.csv")
df_different2 = df_emb2[df_emb2["Zweckbestimmung"] != df_emb2["mapped_idx"]]
for i in range(0,9):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031 02 und 031 04 erfasste Aufkommen)' ,
'Energiesteuer (sonstiges Aufkommen, ohne das in den Titeln 031\xa002 und 031\xa004 erfasste Aufkommen)'

'Arbeitslosengeld II' ,
'Entgelte der Arbeitnehmerinnen und Arbeitnehmer'

'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamt gesellschaftliche Aufgaben (Gesundheitsfonds)' ,
'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamtgesellschaftliche Aufgaben (Gesundheitsfonds)'

'Nicht veranlagte Steuern vom Ertrag (ohne das in Tit. 018 03 erfasste Aufkommen)' ,
'Nicht veranlagte Steuern vom Ertrag (ohne das in Tit.\xa0018\xa003 erfasste Aufkommen)'

'Beteiligung am Grundkapital des Europäischen Stabilitätsmechanismus (ESM)' ,
'Zuschüsse zu Vorhaben zur Förderung des europäischen Gedankens'

'Beteiligung des Bundes an den Kosten der Arbeitsförderung' ,
'Erstattung von Verwaltungskosten an die Bundesagentur für Arbeit für die Durchführung des Bundeskindergeld

In [34]:
for i in range(9,18):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Beteiligung des Bundes an der Grundsicherung im Alter und bei Erwerbsminderung' ,
'Erstattungen des Bundes für die Grundsicherung im Alter und bei Erwerbsminderung'

'Bilaterale Finanzielle Zusammenarbeit' ,
'Zinsen aus Darlehen der bilateralen Finanziellen Zusammenarbeit und Erträge aus Treuhandbeteiligungen'

'Erhöhung des Kapitalanteils an der Europäischen Investitionsbank' ,
'Altersentschädigung an ausgeschiedene Mitglieder des Europäischen Parlaments sowie Hinterbliebenenversorgung nach §\xa010\xa0b Europaabgeordnetengesetz'

'Kernbrennstoffsteuer' ,
'Kaffeesteuer'

'Hochschulpakt 2020' ,
'StIL - Betrieb'

'Kompensationszahlungen an die Länder wegen Beendigung der Finanzhilfen des Bundes für Investitionen zur Verbesserung der Verkehrsverhältnisse der Gemeinden' ,
'Zuweisungen für Investitionen an die Länder'

'Zinsen gem. § 4 des Schlusszahlungsfinanzierungsgesetzes (SchlussFinG)' ,
'Zinsen gem. §\xa04 des Schlusszahlungsfinanzierungsgesetzes (SchlussFinG)'

'Baukostenzuschüsse f

In [35]:
df_emb2.shape[0], df_emb2["mapped_idx"].isna().sum()

(5401, np.int64(1808))

In [21]:
df_emb2 = pd.read_csv("data/vergleich_HR2012_s2_80_ist_test2.csv")

In [22]:
df_emb2["Ist 2012"] = df_emb2["Ist 2012"].round(-3).div(1000).astype(int).astype(str) #.iloc[0]

In [23]:
df_emb2

,Unnamed: 0,index,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,id,id_nlp_help,id_nlp,mapped_idx
0,3,3078,11,13,63681,31569893,Zuschuss des Bundes an die allgemeine Rentenve...,636811113,63681111331569893,sechsdreisechsachteinseinseinseinsdreidreieins...,Zuschuss des Bundes an die allgemeine Rentenve...
1,6,5252,32,1,32511,22480026,Einnahmen aus Krediten vom Kreditmarkt,325113201,32511320122480026,dreizweifünfeinseinsdreizweinulleinszweizweivi...,Einnahmen aus Krediten vom Kreditmarkt
2,7,3080,11,13,63683,20122813,Zusätzlicher Zuschuss des Bundes an die allgem...,636831113,63683111320122813,sechsdreisechsachtdreieinseinseinsdreizweinull...,Zusätzlicher Zuschuss des Bundes an die allgem...
3,8,3058,11,12,68112,18951336,Arbeitslosengeld II,681121112,68112111218951336,sechsachteinseinszweieinseinseinszweieinsachtn...,Entgelte der Arbeitnehmerinnen und Arbeitnehmer
4,11,4267,15,2,63606,14000000,Pauschale Abgeltung der Aufwendungen der Krank...,636061502,63606150214000000,sechsdreisechsnullsechseinsfünfnullzweieinsvie...,Pauschale Abgeltung der Aufwendungen der Krank...
...,...,...,...,...,...,...,...,...,...,...,...
4383,5284,52,1,4,52901,3,Außergewöhnlicher Aufwand aus dienstlicher Ver...,529010104,5290101043,fünfzweineunnulleinsnulleinsnullvierdrei Auße...,Außergewöhnlicher Aufwand aus dienstlicher Ver...
4384,5285,2977,11,7,11999,3,Vermischte Einnahmen,119991107,1199911073,einseinsneunneunneuneinseinsnullsiebendrei Ve...,Vermischte Einnahmen
4385,5286,190,3,1,44301,3,Fürsorgeleistungen und Unterstützungen,443010301,4430103013,viervierdreinulleinsnulldreinulleinsdrei Fürs...,NaN
4386,5287,3919,14,1,52602,3,Sachverständige,526021401,5260214013,fünfzweisechsnullzweieinsviernulleinsdrei Sac...,NaN


In [24]:
df_emb2

,Unnamed: 0,index,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,id,id_nlp_help,id_nlp,mapped_idx
0,3,3078,11,13,63681,31569893,Zuschuss des Bundes an die allgemeine Rentenve...,636811113,63681111331569893,sechsdreisechsachteinseinseinseinsdreidreieins...,Zuschuss des Bundes an die allgemeine Rentenve...
1,6,5252,32,1,32511,22480026,Einnahmen aus Krediten vom Kreditmarkt,325113201,32511320122480026,dreizweifünfeinseinsdreizweinulleinszweizweivi...,Einnahmen aus Krediten vom Kreditmarkt
2,7,3080,11,13,63683,20122813,Zusätzlicher Zuschuss des Bundes an die allgem...,636831113,63683111320122813,sechsdreisechsachtdreieinseinseinsdreizweinull...,Zusätzlicher Zuschuss des Bundes an die allgem...
3,8,3058,11,12,68112,18951336,Arbeitslosengeld II,681121112,68112111218951336,sechsachteinseinszweieinseinseinszweieinsachtn...,Entgelte der Arbeitnehmerinnen und Arbeitnehmer
4,11,4267,15,2,63606,14000000,Pauschale Abgeltung der Aufwendungen der Krank...,636061502,63606150214000000,sechsdreisechsnullsechseinsfünfnullzweieinsvie...,Pauschale Abgeltung der Aufwendungen der Krank...
...,...,...,...,...,...,...,...,...,...,...,...
4383,5284,52,1,4,52901,3,Außergewöhnlicher Aufwand aus dienstlicher Ver...,529010104,5290101043,fünfzweineunnulleinsnulleinsnullvierdrei Auße...,Außergewöhnlicher Aufwand aus dienstlicher Ver...
4384,5285,2977,11,7,11999,3,Vermischte Einnahmen,119991107,1199911073,einseinsneunneunneuneinseinsnullsiebendrei Ve...,Vermischte Einnahmen
4385,5286,190,3,1,44301,3,Fürsorgeleistungen und Unterstützungen,443010301,4430103013,viervierdreinulleinsnulldreinulleinsdrei Fürs...,NaN
4386,5287,3919,14,1,52602,3,Sachverständige,526021401,5260214013,fünfzweisechsnullzweieinsviernulleinsdrei Sac...,NaN


In [27]:
df_emb2 = pd.read_csv("data/vergleich_HR2012_s2_80_with_ist.csv")
df_different2 = df_emb2[df_emb2["Zweckbestimmung"] != df_emb2["mapped_idx"]]
print(df_emb2.shape[0], df_emb2["mapped_idx"].isna().sum())

4388 1952


In [28]:
for i in range(0,9):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamt gesellschaftliche Aufgaben (Gesundheitsfonds)' ,
'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamtgesellschaftliche Aufgaben (Gesundheitsfonds)'

'Beteiligung am Grundkapital des Europäischen Stabilitätsmechanismus (ESM)' ,
nan

'Beteiligung des Bundes an den Kosten der Arbeitsförderung' ,
nan

'Einnahmen aus der streckenbezogenen LKW-Maut' ,
'Einnahmen aus der streckenbezogenen Maut'

'Eingliederungsbeitrag der Bundesagentur für Arbeit' ,
nan

'Beteiligung des Bundes an der Grundsicherung im Alter und bei Erwerbsminderung' ,
'Erstattungen des Bundes für die Grundsicherung im Alter und bei Erwerbsminderung'

'Erstattungen für Aufwendungen aufgrund der Überführung von Zusatzversorgungssystemen in die Rentenversicherung' ,
nan

'Bilaterale Finanzielle Zusammenarbeit' ,
'Bilaterale Finanzielle Zusammenarbeit - Darlehen'

'Erhöhung des Kapitalanteils an der Europäischen Investitionsbank' ,
nan



In [29]:
for i in range(9,18):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Kernbrennstoffsteuer' ,
nan

'Hochschulpakt 2020' ,
nan

'Kompensationszahlungen an die Länder wegen Beendigung der Finanzhilfen des Bundes für Investitionen zur Verbesserung der Verkehrsverhältnisse der Gemeinden' ,
nan

'Baukostenzuschüsse für Investitionen in die Schienenwege der Eisenbahnen des Bundes' ,
'Baukostenzuschüsse für Investitionen in die Schienenwege der nicht bundeseigenen Eisenbahnen'

'Zuschüsse an die Träger der Krankenversicherung der Landwirte' ,
nan

'Erhaltung (Bundesautobahnen)' ,
nan

'Zuschüsse für den Absatz deutscher Steinkohle zur Verstromung, zum Absatz an die Stahlindustrie sowie zum Ausgleich von Belastungen infolge von Kapazitätsanpassungen' ,
nan

'Zuschüsse zu den Beiträgen zur Rentenversicherung der in Werkstätten und Integrationsprojekten beschäftigten behinderten Menschen' ,
'Zuschüsse zu den Beiträgen zur Rentenversicherung der in Werkstätten, bei anderen Leistungsanbietern nach § 60 SGB IX und Inklusionsbetrieben beschäftigten behinderten Mensch

In [26]:
for i in range(18,27):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Versorgungsbezüge für Beschädigte und Hinterbliebene' ,
nan

'Förderung von Maßnahmen zur energetischen Gebäudesanierung "CO2- Gebäudesanierungsprogramm" der KfW-Bankengruppe - Abwicklung' ,
nan

'Bedarfsplanmaßnahmen (Bundesautobahnen)' ,
nan

'Kompensationsmittel für die Abschaffung der GA Hochschulbau' ,
nan

'Anteil des Bundes am Reingewinn der Deutschen Bundesbank' ,
nan

'Ausgaben für Aufträge und Dienstleistungen' ,
nan

'Beitrag zu den "Europäischen Entwicklungsfonds" der Europäischen Union (Abkommen von Lom  und Cotonou)' ,
'Beitrag zu den "Europäischen Entwicklungsfonds" der Europäischen Union (Abkommen von Lomé und Cotonou)'

'Zuweisungen für Investitionen an das Sondervermögen Kinderbetreuungsausbau für die "Kinderbetreuungsfinanzierung 2013-2014"' ,
nan

'Kompensationszahlungen an die Länder wegen Beendigung der Finanzhilfen des Bundes zur Sozialen Wohnraumförderung' ,
nan



In [ ]:
df_emb2 = pd.read_csv("data/vergleich_HR2012_s2_80_ist_test3.csv")
df_different2 = df_emb2[df_emb2["Zweckbestimmung"] != df_emb2["mapped_idx"]]
print(df_emb2.shape[0], df_emb2["mapped_idx"].isna().sum())

In [11]:
for i in range(0,9):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamt gesellschaftliche Aufgaben (Gesundheitsfonds)' ,
'Pauschale Abgeltung der Aufwendungen der Krankenkassen für gesamtgesellschaftliche Aufgaben (Gesundheitsfonds)'

'Beteiligung am Grundkapital des Europäischen Stabilitätsmechanismus (ESM)' ,
nan

'Beteiligung des Bundes an den Kosten der Arbeitsförderung' ,
'Erstattung von Verwaltungskosten an die Bundesagentur für Arbeit für die Durchführung des Bundeskindergeldgesetzes'

'Einnahmen aus der streckenbezogenen LKW-Maut' ,
'Einnahmen aus der streckenbezogenen Maut'

'Eingliederungsbeitrag der Bundesagentur für Arbeit' ,
'Rückzahlung des Darlehens durch die Bundesagentur für Arbeit'

'Beteiligung des Bundes an der Grundsicherung im Alter und bei Erwerbsminderung' ,
'Erstattungen des Bundes für die Grundsicherung im Alter und bei Erwerbsminderung'

'Erstattungen für Aufwendungen aufgrund der Überführung von Zusatzversorgungssystemen in die Rentenversicherung' ,
nan

'Bilater

In [12]:
for i in range(9,18):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Kernbrennstoffsteuer' ,
nan

'Hochschulpakt 2020' ,
nan

'Kompensationszahlungen an die Länder wegen Beendigung der Finanzhilfen des Bundes für Investitionen zur Verbesserung der Verkehrsverhältnisse der Gemeinden' ,
nan

'Baukostenzuschüsse für Investitionen in die Schienenwege der Eisenbahnen des Bundes' ,
'Baukostenzuschüsse für Investitionen in die Schienenwege der nicht bundeseigenen Eisenbahnen'

'Zuschüsse an die Träger der Krankenversicherung der Landwirte' ,
'Zuschüsse zur Krankenversicherung der Landwirte'

'Erhaltung (Bundesautobahnen)' ,
'Erhaltung (Bundesstraßen)'

'Zuschüsse für den Absatz deutscher Steinkohle zur Verstromung, zum Absatz an die Stahlindustrie sowie zum Ausgleich von Belastungen infolge von Kapazitätsanpassungen' ,
nan

'Zuschüsse zu den Beiträgen zur Rentenversicherung der in Werkstätten und Integrationsprojekten beschäftigten behinderten Menschen' ,
'Zuschüsse zu den Beiträgen zur Rentenversicherung der in Werkstätten, bei anderen Leistungsanbietern nac

In [13]:
for i in range(18,27):
    print(repr(df_different2.iloc[i]["Zweckbestimmung"]), ",\n"+repr(df_different2.iloc[i]["mapped_idx"])+"\n")

'Versorgungsbezüge für Beschädigte und Hinterbliebene' ,
nan

'Förderung von Maßnahmen zur energetischen Gebäudesanierung "CO2- Gebäudesanierungsprogramm" der KfW-Bankengruppe - Abwicklung' ,
nan

'Bedarfsplanmaßnahmen (Bundesautobahnen)' ,
'Investitionszuschuss an die "Die Autobahn GmbH des Bundes" für Bedarfsplanmaßnahmen gemäß Anlage 5 Investitionsgesetz Kohleregionen - InvKG (Bundesautobahnen)'

'Kompensationsmittel für die Abschaffung der GA Hochschulbau' ,
nan

'Anteil des Bundes am Reingewinn der Deutschen Bundesbank' ,
'Erstattung der Bundesbank für den Objektschutz durch die Bundespolizei'

'Beitrag zu den "Europäischen Entwicklungsfonds" der Europäischen Union (Abkommen von Lom  und Cotonou)' ,
'Beitrag zu den "Europäischen Entwicklungsfonds" der Europäischen Union (Abkommen von Lomé und Cotonou)'

'Zuweisungen für Investitionen an das Sondervermögen Kinderbetreuungsausbau für die "Kinderbetreuungsfinanzierung 2013-2014"' ,
nan

'Erhaltung (Bundesstraßen)' ,
'Bau von Radwegen

In [93]:
check_mapping_list = [None]*len(df_different2)
for i in range(len(df_different2)):
    check_mapping_list[i] = str(df_different2.iloc[i]["Zweckbestimmung"])+",\n"+str(df_different2.iloc[i]["mapped_idx"])+"\n"

with open('check_mapping_80.txt', 'w', encoding="utf8") as f:
    for line in check_mapping_list:
        f.write(f"{line}\n")

In [98]:
df_different2[df_different2["Zweckbestimmung"].str.contains("Einzelfall")]

,Unnamed: 0,index,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,id,id_nlp_help,id_nlp,mapped_idx
132,203,5242,30,4,89471,1.886386e+08,HGF-Zentren - Investitionen von mehr als 2 500...,894713004,894713004,achtneunviersiebeneinsdreinullnullvier HGF-Ze...,NaN
344,505,4479,15,11,71201,3.667082e+07,Baumaßnahmen von mehr als \n1 000 000 € im Ein...,712011511,712011511,siebeneinszweinulleinseinsfünfeinseins Baumaß...,Baumaßnahmen von mehr als 6 000 000 € im Einze...
371,541,2084,9,5,89431,3.318000e+07,Deutsches Zentrum für Luft- und Raumfahrt e. V...,894310905,894310905,achtneunvierdreieinsnullneunnullfünf Deutsche...,Deutsches Zentrum für Luft- und Raumfahrt e. V...
390,565,2670,10,14,71201,2.944307e+07,Baumaßnahmen von mehr als \n1 000 000 € im Ein...,712011014,712011014,siebeneinszweinulleinseinsnulleinsvier Baumaß...,Baumaßnahmen von mehr als 6 000 000 € im Einze...
418,606,2085,9,5,89432,2.653500e+07,Deutsches Zentrum für Luft- und Raumfahrt e. V...,894320905,894320905,achtneunvierdreizweinullneunnullfünf Deutsche...,Kostenerstattung für das Deutsche Zentrum für ...
455,654,3456,12,10,81212,2.269569e+07,Erwerb von Geräten (einschl. Stahlflachstraßen...,812121210,812121210,achteinszweieinszweieinszweieinsnull Erwerb v...,Erwerb von Geräten und Maschinen mit Ausgaben ...
617,867,3457,12,10,81222,1.274319e+07,Erwerb von Geräten und Maschinen mit Ausgaben ...,812221210,812221210,achteinszweizweizweieinszweieinsnull Erwerb v...,NaN
640,895,869,6,8,71203,1.198864e+07,Baumaßnahmen des Hochbaus im Inland von mehr a...,712030608,712030608,siebeneinszweinulldreinullsechsnullacht Bauma...,Baumaßnahmen des Hochbaus im Ausland von mehr ...
670,946,1181,6,25,71201,1.059961e+07,Baumaßnahmen von mehr als \n1 000 000 € im Ein...,712010625,712010625,siebeneinszweinulleinsnullsechszweifünf Bauma...,Baumaßnahmen von mehr als 6 000 000 € im Einze...
736,1036,3250,12,3,71211,8.495230e+06,Baumaßnahmen von mehr als \n1 000 000 € im Ein...,712111203,712111203,siebeneinszweieinseinseinszweinulldrei Baumaß...,Baumaßnahmen von mehr als 6 000 000 € im Einze...


In [96]:
df_different2["mapped_idx"].value_counts()

mapped_idx
Geschäftsbedarf und Kommunikation sowie Geräte, Ausstattungs- und Ausrüstungsgegenstände, sonstige Gebrauchsgegenstände, Software, Wartung    55
Erwerb von Geräten, Ausstattungs- und Ausrüstungsgegenständen für Verwaltungszwecke (ohne IT)                                                 34
Baumaßnahmen von mehr als 6 000 000 € im Einzelfall                                                                                           16
Sachverständige, Ausgaben für Mitglieder von Fachbeiräten und ähnlichen Ausschüssen                                                           16
Erstattungen des Bundes für Versorgungslasten                                                                                                 14
                                                                                                                                              ..
Mitgliedsbeiträge und sonstige Zuschüsse für laufende Zwecke im Inland geringeren Umfangs                              

In [5]:
df_mappped = pd.read_csv("data/vergleich_HR2012_s2_80_ist_test2.csv")
df_10y = pd.read_csv("data/HR10y_on_id.csv")

In [7]:
len(df_mappped)

4388

In [8]:
df_10y.dtypes

Unnamed: 0           int64
id                   int64
Zweckbestimmung     object
Ist 2012           float64
Ist 2013           float64
Ist 2014           float64
Ist 2015           float64
Ist 2016           float64
Ist 2017           float64
Ist 2018           float64
Ist 2019           float64
Ist 2020           float64
Ist 2021           float64
Ist 2022           float64
Ist 2023           float64
dtype: object

In [9]:
df_mappped.dtypes

Unnamed: 0           int64
Epl.                 int64
Kap.                 int64
Tit.                 int64
Ist 2012           float64
Zweckbestimmung     object
id                   int64
id_nlp_help          int64
id_nlp              object
mapped_idx          object
dtype: object

In [10]:
df_unmatch = df_mappped[~df_mappped["id"].isin(df_10y["id"])]

In [11]:
df_unmatch

,Unnamed: 0,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,id,id_nlp_help,id_nlp,mapped_idx
3,3078,11,13,63681,3.156989e+10,Zuschuss des Bundes an die allgemeine Rentenve...,636811113,63681111331569893,sechsdreisechsachteinseinseinseinsdreidreieins...,Zuschuss des Bundes an die allgemeine Rentenve...
6,5252,32,1,32511,2.248003e+10,Einnahmen aus Krediten vom Kreditmarkt,325113201,32511320122480026,dreizweifünfeinseinsdreizweinulleinszweizweivi...,Einnahmen aus Krediten vom Kreditmarkt
7,3080,11,13,63683,2.012281e+10,Zusätzlicher Zuschuss des Bundes an die allgem...,636831113,63683111320122813,sechsdreisechsachtdreieinseinseinsdreizweinull...,Zusätzlicher Zuschuss des Bundes an die allgem...
8,3058,11,12,68112,1.895134e+10,Arbeitslosengeld II,681121112,68112111218951336,sechsachteinseinszweieinseinseinszweieinsachtn...,Entgelte der Arbeitnehmerinnen und Arbeitnehmer
11,4267,15,2,63606,1.400000e+10,Pauschale Abgeltung der Aufwendungen der Krank...,636061502,63606150214000000,sechsdreisechsnullsechseinsfünfnullzweieinsvie...,Pauschale Abgeltung der Aufwendungen der Krank...
...,...,...,...,...,...,...,...,...,...,...
5396,838,6,7,81202,1.099070e+03,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...",812020607,8120206071,achteinszweinullzweinullsechsnullsiebeneins E...,NaN
5397,1436,6,67,45357,1.084700e+03,"Trennungsgeld, Fahrtkostenzuschüsse sowie Umzu...",453570667,4535706671,vierfünfdreifünfsiebennullsechssechssiebeneins...,"Trennungsgeld, Fahrtkostenzuschüsse sowie Umzu..."
5398,1956,8,14,67101,1.076000e+03,Aufwendungen für Besucherfahrten zu Gräbern un...,671010814,6710108141,sechssiebeneinsnulleinsnullachteinsviereins A...,NaN
5399,1406,6,40,16204,1.034430e+03,Zinsen aus Darlehen an Deutsche aus dem in Art...,162040640,1620406401,einssechszweinullviernullsechsviernulleins Zi...,NaN


In [77]:
arbeit_idx = df_unmatch[df_unmatch["Zweckbestimmung"] == "Arbeitslosengeld II"].index
df_unmatch.loc[arbeit_idx[0],"Zweckbestimmung"] = "Arbeitslosengeld I"

In [78]:
df_unmatch

,Unnamed: 0,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,id,id_nlp_help,id_nlp,mapped_idx
3,3078,11,13,63681,3.156989e+10,Zuschuss des Bundes an die allgemeine Rentenve...,636811113,63681111331569893,sechsdreisechsachteinseinseinseinsdreidreieins...,Zuschuss des Bundes an die allgemeine Rentenve...
6,5252,32,1,32511,2.248003e+10,Einnahmen aus Krediten vom Kreditmarkt,325113201,32511320122480026,dreizweifünfeinseinsdreizweinulleinszweizweivi...,Einnahmen aus Krediten vom Kreditmarkt
7,3080,11,13,63683,2.012281e+10,Zusätzlicher Zuschuss des Bundes an die allgem...,636831113,63683111320122813,sechsdreisechsachtdreieinseinseinsdreizweinull...,Zusätzlicher Zuschuss des Bundes an die allgem...
8,3058,11,12,68112,1.895134e+10,Arbeitslosengeld I,681121112,68112111218951336,sechsachteinseinszweieinseinseinszweieinsachtn...,Entgelte der Arbeitnehmerinnen und Arbeitnehmer
11,4267,15,2,63606,1.400000e+10,Pauschale Abgeltung der Aufwendungen der Krank...,636061502,63606150214000000,sechsdreisechsnullsechseinsfünfnullzweieinsvie...,Pauschale Abgeltung der Aufwendungen der Krank...
...,...,...,...,...,...,...,...,...,...,...
5396,838,6,7,81202,1.099070e+03,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...",812020607,8120206071,achteinszweinullzweinullsechsnullsiebeneins E...,NaN
5397,1436,6,67,45357,1.084700e+03,"Trennungsgeld, Fahrtkostenzuschüsse sowie Umzu...",453570667,4535706671,vierfünfdreifünfsiebennullsechssechssiebeneins...,"Trennungsgeld, Fahrtkostenzuschüsse sowie Umzu..."
5398,1956,8,14,67101,1.076000e+03,Aufwendungen für Besucherfahrten zu Gräbern un...,671010814,6710108141,sechssiebeneinsnulleinsnullachteinsviereins A...,NaN
5399,1406,6,40,16204,1.034430e+03,Zinsen aus Darlehen an Deutsche aus dem in Art...,162040640,1620406401,einssechszweinullviernullsechsviernulleins Zi...,NaN


In [8]:
        #test = pd.DataFrame(result_all).value_counts()
        #print(test)
        # test2 = pd.DataFrame(result_all)[0].isna().sum()
        # print("NaN number:", test2,"all_rows:", len(df_i))

In [9]:
for year in range(12,24):
    df_i = pd.read_csv(f"data/HR20{year}_nlp_mapped.csv")
    test2 = pd.DataFrame(df_i)["id"].isna().sum()
    print(f"Year: {year}",", NaN number:", test2,", all_rows:", len(df_i))

Year: 12 , NaN number: 1952 , all_rows: 4388
Year: 13 , NaN number: 1766 , all_rows: 4317
Year: 14 , NaN number: 1146 , all_rows: 3994
Year: 15 , NaN number: 1132 , all_rows: 4026
Year: 16 , NaN number: 757 , all_rows: 3928
Year: 17 , NaN number: 733 , all_rows: 4000
Year: 18 , NaN number: 683 , all_rows: 3989
Year: 19 , NaN number: 687 , all_rows: 4067
Year: 20 , NaN number: 632 , all_rows: 4155
Year: 21 , NaN number: 609 , all_rows: 4235
Year: 22 , NaN number: 551 , all_rows: 4338
Year: 23 , NaN number: 1 , all_rows: 4388


In [15]:
for year in range(23,11, -1):
    df_i = pd.read_csv(f"data/HR20{year}_nlp_mapped.csv")
    test2 = pd.DataFrame(df_i)["id"].isna().sum()
    print(f"Year: {year}",", NaN number:", test2,", all_rows:", len(df_i))

Year: 23 , NaN number: 1 , all_rows: 4388
Year: 22 , NaN number: 551 , all_rows: 4338
Year: 21 , NaN number: 609 , all_rows: 4235
Year: 20 , NaN number: 632 , all_rows: 4155
Year: 19 , NaN number: 687 , all_rows: 4067
Year: 18 , NaN number: 683 , all_rows: 3989
Year: 17 , NaN number: 733 , all_rows: 4000
Year: 16 , NaN number: 757 , all_rows: 3928
Year: 15 , NaN number: 1132 , all_rows: 4026
Year: 14 , NaN number: 1146 , all_rows: 3994
Year: 13 , NaN number: 1766 , all_rows: 4317
Year: 12 , NaN number: 1952 , all_rows: 4388


In [18]:
df_dict = df_i.to_dict()

In [26]:
import json
with open("df_test.json", "w") as file:
    json.dump(df_dict, file)

In [20]:
df_dict.keys()

dict_keys(['Unnamed: 0', 'index', 'Epl.', 'Kap.', 'Tit.', 'Ist 2012', 'Zweckbestimmung', '2012 id', 'id_nlp_help', 'id_nlp', '2012 Zweck', 'id'])

In [21]:
df_i

,Unnamed: 0,index,Epl.,Kap.,Tit.,Ist 2012,Zweckbestimmung,2012 id,id_nlp_help,id_nlp,2012 Zweck,id
0,3,3078,11,13,63681,3.156989e+10,Zuschuss des Bundes an die allgemeine Rentenve...,636811113,63681111331569893,sechsdreisechsachteinseinseinseinsdreidreieins...,Zuschuss des Bundes an die allgemeine Rentenve...,636811102.0
1,6,5252,32,1,32511,2.248003e+10,Einnahmen aus Krediten vom Kreditmarkt,325113201,32511320122480026,dreizweifünfeinseinsdreizweinulleinszweizweivi...,Einnahmen aus Krediten vom Kreditmarkt,325113201.0
2,7,3080,11,13,63683,2.012281e+10,Zusätzlicher Zuschuss des Bundes an die allgem...,636831113,63683111320122813,sechsdreisechsachtdreieinseinseinsdreizweinull...,Zusätzlicher Zuschuss des Bundes an die allgem...,636831102.0
3,8,3058,11,12,68112,1.895134e+10,Arbeitslosengeld II,681121112,68112111218951336,sechsachteinseinszweieinseinseinszweieinsachtn...,Arbeitslosengeld II,681121101.0
4,11,4267,15,2,63606,1.400000e+10,Pauschale Abgeltung der Aufwendungen der Krank...,636061502,63606150214000000,sechsdreisechsnullsechseinsfünfnullzweieinsvie...,Pauschale Abgeltung der Aufwendungen der Krank...,636061501.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4383,5284,52,1,4,52901,2.849440e+03,Außergewöhnlicher Aufwand aus dienstlicher Ver...,529010104,5290101043,fünfzweineunnulleinsnulleinsnullvierdrei Auße...,Außergewöhnlicher Aufwand aus dienstlicher Ver...,529010711.0
4384,5285,2977,11,7,11999,2.821450e+03,Vermischte Einnahmen,119991107,1199911073,einseinsneunneunneuneinseinsnullsiebendrei Ve...,Vermischte Einnahmen,119991912.0
4385,5286,190,3,1,44301,2.811960e+03,Fürsorgeleistungen und Unterstützungen,443010301,4430103013,viervierdreinulleinsnulldreinulleinsdrei Fürs...,NaN,NaN
4386,5287,3919,14,1,52602,2.808400e+03,Sachverständige,526021401,5260214013,fünfzweisechsnullzweieinsviernulleinsdrei Sac...,NaN,NaN
